## Accuracy: 0.496

In [1]:
import numpy as np
import random
from gensim.models import word2vec

In [2]:
programs_questions = []
num_lines = 0
with open('program_question.txt', 'r') as inFile:
    for line in inFile:
        programs_questions.append(line.strip('\n'))
        num_lines += 1
        
        if num_lines % 100000 == 0:
            print ("Finished %d of lines" % num_lines)
            
word2vec_model = word2vec.Word2Vec.load('word2vec_model.bin')

Finished 100000 of lines
Finished 200000 of lines
Finished 300000 of lines
Finished 400000 of lines
Finished 500000 of lines
Finished 600000 of lines
Finished 700000 of lines
Finished 800000 of lines
Finished 900000 of lines
Finished 1000000 of lines
Finished 1100000 of lines
Finished 1200000 of lines
Finished 1300000 of lines
Finished 1400000 of lines
Finished 1500000 of lines
Finished 1600000 of lines
Finished 1700000 of lines
Finished 1800000 of lines
Finished 1900000 of lines
Finished 2000000 of lines
Finished 2100000 of lines
Finished 2200000 of lines
Finished 2300000 of lines
Finished 2400000 of lines


In [3]:
programs = programs_questions[:2396906]
questions = programs_questions[2396906:]

In [4]:
def generate_training_data():
    Xs, Ys = [], []
    NUM_TRAIN = 300000
    itr = 0
    
    while (itr < NUM_TRAIN):
        pos_or_neg = random.randint(0, 1)
        
        if (pos_or_neg is 1):
            line_id = random.randint(1, len(programs)-1)
            
            L1 = programs[line_id].split(' ')[:-1]
            L2 = programs[line_id+1].split(' ')[:-1]
            L3 = programs[line_id-1].split(' ')[:-1]
            
            if (L1 and L2 and L3):
                try:
                    sim1 = word2vec_model.n_similarity(L1, L2)
                    sim2 = word2vec_model.n_similarity(L1, L3)
                    if (sim1 > sim2):
                        Ys.append(1)
                        Xs.append(programs[line_id])
                        Xs.append(programs[line_id+1])
                    else:
                        Ys.append(1)
                        Xs.append(programs[line_id])
                        Xs.append(programs[line_id-1])
                    itr += 1
                except KeyError:
                    pass
                
        else:
            line1_id = random.randint(0, len(programs)-1)
            line2_id = random.randint(0, len(programs)-1)
            
            L1 = programs[line1_id].split(' ')[:-1]
            L2 = programs[line2_id].split(' ')[:-1]
            
            if (L1 and L2):
                try:
                    Ys.append(-1)
                    Xs.append(programs[line1_id])
                    Xs.append(programs[line2_id])
                    itr += 1
                except KeyError:
                    pass
                
    return Xs, Ys
            

In [5]:
data, y = generate_training_data()

for question in questions:
    data.append(question)


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(3,4), sublinear_tf=True)
tfidf.fit(data)

doc_tf = tfidf.transform(data).toarray()

In [7]:
doc_vec = []

for sentence, tf_idf in zip(data, doc_tf):
    words = sentence.split(' ')[:-1]
    vec = np.zeros(shape=(100,), dtype=float)
    
    for w in words:
        try:
            vec = np.add(vec, word2vec_model.wv[w]*tf_idf[tfidf.vocabulary_[w]])
        except KeyError:
            vec = np.add(vec, word2vec_model.wv[w]*0.000001)
        
    doc_vec.append(vec)

In [8]:
programs_vec = doc_vec[:600000]
questions_vec = doc_vec[600000:]

In [9]:
print (len(programs_vec))
print (len(questions_vec))

600000
3500


In [10]:
import math
X_ = []

for _id in range(300000):
    X_.append(np.dot(programs_vec[_id*2], programs_vec[_id*2+1])*programs_vec[_id*2]/np.linalg.norm(programs_vec[_id*2]))
    
X_ = np.stack(X_, axis=0)
y = np.array(y)

#for i, vec in enumerate(X_):
#    for item in vec:
#        x = float(item)
#        if math.isnan(x):
#            print (i)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

X_train, X_valid, y_train, y_valid = train_test_split(X_, y, test_size=0.3, random_state=1)
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_valid_std = sc.transform(X_valid)

In [ ]:
from sklearn.linear_model import Lasso

a = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
best_mse = 100000
best_a = -1

for alpha in a:
    las = Lasso(alpha=alpha)
    
    las.fit(X_train_std, y_train)
    y_pred = las.predict(X_valid_std)
    
    if (mean_squared_error(y_valid, y_pred) < best_mse):
        best_mse = mean_squared_error(y_valid, y_pred)
        best_a = alpha
        
print ("Best MSE: %.4f with alpha: %.4f" % (best_mse, best_a))
    

In [ ]:
las = Lasso(alpha=0.0001)

las.fit(X_train_std, y_train)
y_pred_las = las.predict(X_valid_std)

print ("MSE: %.4f" % (mean_squared_error(y_valid, y_pred_las)))

In [12]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_std, y_train)
y_pred_lr = lr.predict(X_valid_std)

print ("MSE: %.4f" % (mean_squared_error(y_valid, y_pred_lr)))

/Users/larry/Py3/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


MSE: 0.8245


In [14]:
import sys

q_test = []
for q_id in range(500):
    question = questions_vec[q_id*7]
    answer = 0
    sim_max = sys.float_info.min
    for i in range(1,7):
        ans = questions_vec[q_id*7 + i]
        #set_ = [x+y for x, y in zip(question, ans)]
        set_ = np.dot(question, ans)*question/np.linalg.norm(question)
        #set_ = np.add(question, ans)
        result = lr.predict(set_.reshape(1,-1))
        if result > sim_max:
            answer = i-1
            sim_max = result
        
    q_test.append((q_id, answer))

In [15]:
print (q_test)

[(0, 4), (1, 0), (2, 3), (3, 4), (4, 3), (5, 4), (6, 2), (7, 1), (8, 0), (9, 4), (10, 0), (11, 2), (12, 4), (13, 0), (14, 2), (15, 2), (16, 4), (17, 4), (18, 0), (19, 4), (20, 2), (21, 2), (22, 3), (23, 0), (24, 3), (25, 1), (26, 4), (27, 5), (28, 4), (29, 2), (30, 2), (31, 2), (32, 5), (33, 5), (34, 4), (35, 3), (36, 4), (37, 4), (38, 1), (39, 1), (40, 0), (41, 4), (42, 5), (43, 0), (44, 3), (45, 5), (46, 5), (47, 2), (48, 2), (49, 3), (50, 1), (51, 3), (52, 5), (53, 3), (54, 1), (55, 1), (56, 5), (57, 1), (58, 2), (59, 4), (60, 0), (61, 2), (62, 4), (63, 0), (64, 3), (65, 4), (66, 0), (67, 3), (68, 0), (69, 4), (70, 5), (71, 5), (72, 0), (73, 5), (74, 3), (75, 1), (76, 1), (77, 4), (78, 1), (79, 3), (80, 0), (81, 2), (82, 2), (83, 3), (84, 3), (85, 1), (86, 4), (87, 1), (88, 2), (89, 0), (90, 5), (91, 1), (92, 4), (93, 5), (94, 3), (95, 1), (96, 4), (97, 5), (98, 3), (99, 0), (100, 1), (101, 2), (102, 0), (103, 3), (104, 1), (105, 2), (106, 1), (107, 2), (108, 2), (109, 1), (110, 4),

In [16]:
import pandas as pd

df = pd.DataFrame(data=q_test, columns=['Id', 'Answer']).to_csv('Answer.csv', index=False)